# Research Project - Jupyter Notebook
Author: Frank van Mourik <f.g.vanmourik@student.utwente.nl><br/>

## Explanation
This Notebook includes the queries executed on a part of the CT log dataset stored on the UT clusters. These queries lay the basis for the Bachelor Research Project called "Finding personally identifiable information leaked via publicly accessible CT logs"

## Imports & startup

### Imports

In [1]:
import os
import time
from datetime import datetime, timedelta, date
import pprint

import collections
import numpy as np
import pandas as pd
import subprocess

import pyspark
import pyspark.sql.functions as psf
from pyspark.sql.types import *

# Find Spark
import findspark
findspark.init()

### Create Spark Configuration

In [2]:
# Create a SparkConf

# This name will be visible on the Hadoop Cluster webpage
APP_NAME = "pyspark-CT-analysis-research-project"
# On University cluster: do not increase this number above 400. Default: 10. On large dataset (1 billion records): 300 executors
MAX_NR_OF_EXECUTORS = 300

# Set configuration for cluster executions. Variables can be altered to acquire more and less processing power and executors.
spark_conf = pyspark.SparkConf().setAppName(APP_NAME).setMaster("yarn").set(
    "spark.submit.deployMode", "client"
).set("spark.sql.parquet.binaryAsString", "true"
).set("spark.dynamicAllocation.maxExecutors", str(MAX_NR_OF_EXECUTORS)
)

#### Start SparkContext
1. This may take a minute to complete
2. You should not (and cannot) start two Spark contexts. If you accidentally run this cell twice or get stuck somehow, restart your Python kernel from the menu above.
3. Please **stop your Spark context** when idling for extended periods of time

In [3]:
print("[{}] Starting Spark context.".format(datetime.now().replace(microsecond=0)))

# SparkContext
sc = pyspark.SparkContext(conf=spark_conf)

# SQLContext
sqlc = pyspark.SQLContext(sc)
print("SparkContext is running on version ",sc.version)

[2021-06-24 17:03:39] Starting Spark context.
SparkContext is running on version  2.4.7


In [4]:
# The CT data base path on the UT Hadoop cluster
CT_DATA_PATH = "/user/jonkerm/data/ct-logs"
LOG = "Google-Pilot"
# We take number_of_groups batches from the Google Pilot log
number_of_groups = 108 # Do not increase above 108 for Google Pilot!
files = [os.path.join(CT_DATA_PATH, "name="+LOG, "batch-group="+str(x)) for x in range(number_of_groups)]
# print(files)
ct_raw_df = sqlc.read.option("basePath", CT_DATA_PATH).parquet(*files)

NR_RECORDS = ct_raw_df.count()
print("Number of certificates in DF: ", NR_RECORDS)
print("The data schema looks like:")
ct_raw_df.printSchema()

Number of certificates in DF:  1077200000
The data schema looks like:
root
 |-- cert_index: integer (nullable = true)
 |-- cert_link: string (nullable = true)
 |-- chain: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- as_der: string (nullable = true)
 |    |    |-- extensions: struct (nullable = true)
 |    |    |    |-- authorityInfoAccess: string (nullable = true)
 |    |    |    |-- authorityKeyIdentifier: string (nullable = true)
 |    |    |    |-- basicConstraints: string (nullable = true)
 |    |    |    |-- certificatePolicies: string (nullable = true)
 |    |    |    |-- crlDistributionPoints: string (nullable = true)
 |    |    |    |-- keyUsage: string (nullable = true)
 |    |    |    |-- subjectKeyIdentifier: string (nullable = true)
 |    |    |-- fingerprint: string (nullable = true)
 |    |    |-- not_after: float (nullable = true)
 |    |    |-- not_before: float (nullable = true)
 |    |    |-- serial_number: string (nullable = 

## Filter the DF to only include unique Dutch domains

In [5]:
df_dutch_domains = ct_raw_df.select(
    psf.explode('all_domains').alias("domain"),
    psf.col("leaf_cert.not_before").alias("date")
).filter(
    psf.col("domain").contains('.nl')
).select(
    psf.lower(psf.col("domain")).alias("domain"),
    psf.from_unixtime(psf.col("date"), "yyyy-MM-dd hh:mm").alias("date")
).groupby(
    psf.col("domain")
).agg(
    psf.min(psf.col("date")).alias("date")
)
# print("Print schema below:")
# filtered_df.printSchema()
# print("Size of df below:")
# df_dutch_domains.show(10,False)
print("The total number of rows is: ",df_dutch_domains.count())

The total number of rows is:  6404729


In [37]:
# # Possibility to save intermediate results
# timestamp_format = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
# df_dutch_domains.toPandas().to_csv("results/intermediate/{}_{}m-records-all-dutch-domains.csv".format(timestamp_format, str(NR_RECORDS//1000000)))

## Calculate total number of domains in certificates

In [8]:
## HEAVY OPERATION - only use if interested in total number of domains in all certificates
# total_domains = ct_raw_df.select(
#     psf.explode('all_domains').alias("domain")
# )
# print("The total number of certificates are: ",total_domains.count())

# total_unique_domains = total_domains.select(
#     psf.col("domain")
# ).groupby(
#     psf.col("domain")
# ).count()
# print("The total number of domains in the certificates is: ",total_unique_domains.count())


The total number of unique certificates are:  578961023


## Import first names and last names data
- Load the top-100 most-common used last names in the Netherlands, put them to lower-case, and create a string as input for the rlike() method
- Load the top-100 most-common used first names per sex per decade in the Netherlands from 1960-2010, take all unique values, put them to lower-case, and create a string as input for the rlike() method

In [6]:
last_names = [x.lower() for x in pd.read_csv("data/last_names/last_names.csv", names=["lastNames"]).lastNames.to_list()]
print("The total number of last names: ",len(last_names))
last_names_string = '|'.join(last_names)
print("First 100 characters of last names string: ",last_names_string[0:100])

first_names = sqlc.read.option("basePath", "research_project/data/first_names/").csv("research_project/data/first_names/").select(psf.col('_c0').alias("firstNames"), 'sex', 'year')
# first_names.toPandas().to_csv("results/all_first_names.csv")
unique_first_names = first_names.toPandas()['firstNames'].unique()
print("The total number of unique first names: ",len(unique_first_names))
first_names_string = '|'.join([x.lower() for x in unique_first_names])
print("First 100 characters of first names string: ",first_names_string[0:100])

The total number of last names:  102
First 100 characters of last names string:  jong|jansen|vries|berg|dijk|bakker|janssen|visser|smit|meijer|meyer|boer|mulder|groot|bos|vos|peters
The total number of unique first names:  530
First 100 characters of first names string:  maria|johanna|anna|elisabeth|cornelia|wilhelmina|catharina|yvonne|petronella|adriana|hendrika|margar


## Filter domains to include both a common first and a common last name

In [7]:
df_dutch_domains_first_and_last_names = df_dutch_domains.filter(psf.col("domain").rlike(last_names_string)).filter(psf.col("domain").rlike(first_names_string))
df_dutch_domains_first_and_last_names.show(10, False)
print("The number of domains including a first and last name: ", df_dutch_domains_first_and_last_names.count())

In [63]:
agg_per_month = df_dutch_domains_first_and_last_names.withColumn("year", psf.year(psf.col("date"))).withColumn("month", psf.month(psf.col("date"))).select(
    psf.col("year"),
    psf.col("month"),
    psf.col("domain")
).groupBy(
    [psf.col("year"),psf.col("month")]
).count().repartition(1).orderBy(psf.col("year"),psf.col("month"))
# agg_per_month.show()
agg_per_month.toPandas().to_csv("results/name={}/dutch_domain_with_pii_agg_per_month.csv".format(LOG))

In [62]:
print(agg_per_month.count())
# agg_per_month.toPandas().to_csv("results/name={}/dutch_domain_with_pii_agg_per_month.csv".format(LOG))

153


## Split root and four labels closest to the root

In [ ]:
df_split = df_dutch_domains_first_and_last_names.withColumn(
    'split', 
    psf.split(psf.col('domain'),'\\.')
).select(
    psf.element_at(psf.col('split'), -2).alias('root'),
    psf.element_at(psf.col('split'), -3).alias('first_label'),
    psf.element_at(psf.col('split'), -4).alias('second_label'),
    psf.element_at(psf.col('split'), -5).alias('third_label'),
    psf.element_at(psf.col('split'), -6).alias('fourth_label'),
    psf.element_at(psf.col('split'), -7).alias('fifth_label'),
    psf.element_at(psf.col('split'), -8).alias('sixth_label'),
    psf.element_at(psf.col('split'), -9).alias('seventh_label')
)
print("The total number of domains with first and last name is: ",df_split.count())
df_split.show(20,False)

## Count the most-frequent occuring root label

In [42]:
most_occurring_root = df_split.select(
    psf.col("root"),
    psf.col("root").alias("aliass"),
).groupby(
    psf.col("root")
).agg(
    psf.count("aliass").alias("root_count")
).orderBy(psf.col("root_count"), ascending=False)

most_occurring_root.show(20, False)

+-----------------------+----------+
|root                   |root_count|
+-----------------------+----------+
|vpweb                  |636       |
|cas                    |401       |
|amsterdam              |224       |
|fleurglansbeek         |218       |
|vriesencooutdoorliving |195       |
|gcmediavormgeving      |128       |
|s-hertogenbosch        |121       |
|graaggedaan            |111       |
|realworks              |100       |
|probus-nederland       |99        |
|vantlevenproducties    |97        |
|marketheme             |96        |
|kaartenmolen           |94        |
|trimbos                |82        |
|malmberg               |82        |
|sponsoring-events      |76        |
|staatsbosbeheer        |74        |
|md-net                 |72        |
|hoppenbrouwers-techniek|63        |
|martinvanderwal        |61        |
+-----------------------+----------+
only showing top 20 rows



## Check if first or last name in root

In [ ]:
name_string = first_names_string + "|" + last_names_string
pii_in_root = df_split.filter(
    psf.col("root").rlike(name_string)
)
print("The total number of domains with first OR last name in root is: ",pii_in_root.count())
pii_in_root.show(20, False)

## Check if first or last name in root

In [ ]:
first_and_last_name_in_root = df_split.withColumn('pii_in_root',
    psf.when((psf.col("root").rlike(first_names_string) & psf.col("root").rlike(last_names_string)),1).otherwise(0)
).withColumn('pii_in_first_label',
    psf.when((psf.col("first_label").rlike(first_names_string) & psf.col("first_label").rlike(last_names_string)),1).otherwise(0)
).withColumn('pii_in_second_label',
    psf.when((psf.col("second_label").rlike(first_names_string) & psf.col("second_label").rlike(last_names_string)),1).otherwise(0)
).withColumn('pii_in_third_label',
    psf.when((psf.col("third_label").rlike(first_names_string) & psf.col("third_label").rlike(last_names_string)),1).otherwise(0)
).withColumn('pii_in_fourth_label',
    psf.when((psf.col("fourth_label").rlike(first_names_string) & psf.col("fourth_label").rlike(last_names_string)),1).otherwise(0)
).withColumn('pii_in_fifth_label',
    psf.when((psf.col("fifth_label").rlike(first_names_string) & psf.col("fifth_label").rlike(last_names_string)),1).otherwise(0)
).withColumn('pii_in_sixth_label',
    psf.when((psf.col("sixth_label").rlike(first_names_string) & psf.col("sixth_label").rlike(last_names_string)),1).otherwise(0)
).withColumn('pii_in_seventh_label',
    psf.when((psf.col("seventh_label").rlike(first_names_string) & psf.col("seventh_label").rlike(last_names_string)),1).otherwise(0)
)
print("The total number of domains with first AND last name in root is: ",first_and_last_name_in_root.count())
first_and_last_name_in_root.show(20, False)


## Do some computations on binary representation above

In [10]:
sums = first_and_last_name_in_root.groupBy().sum().collect()
print("The total number of occurrences in the root is ",sums[0][0])
print("The total number of occurrences in the first label is ",sums[0][1])
print("The total number of occurrences in the second label is ",sums[0][2])
print("The total number of occurrences in the third label is ",sums[0][3])
print("The total number of occurrences in the fourth label is ",sums[0][4])
print("The total number of occurrences in the fifth label is ",sums[0][5])
print("The total number of occurrences in the sixth label is ",sums[0][6])
print("The total number of occurrences in the seventh label is ",sums[0][7])

The total number of occurrences in the root is  174329
The total number of occurrences in the first label is  5173
The total number of occurrences in the second label is  541
The total number of occurrences in the third label is  300
The total number of occurrences in the fourth label is  12
The total number of occurrences in the fifth label is  2
The total number of occurrences in the sixth label is  0
The total number of occurrences in the seventh label is  0


## Rows with first and last name in root

In [ ]:
data_in_root = first_and_last_name_in_root.select(
    psf.col("root"),
    psf.col("pii_in_root")
).distinct().filter(
    psf.col("pii_in_root") == 1
)
print(data_in_root.count())
data_in_root.show(20,False)

## Rows with first and last name in first label

In [ ]:
data_in_first_label = first_and_last_name_in_root.select(
    psf.col("first_label"),
    psf.col("pii_in_first_label")
).distinct().filter(
    psf.col("pii_in_first_label") == 1
)
print(data_in_first_label.count())
data_in_first_label.show(20,False)

## Rows with first and last name in second label

In [ ]:
data_in_second_label = first_and_last_name_in_root.select(
    psf.col("second_label"),
    psf.col("pii_in_second_label")
).distinct().filter(
    psf.col("pii_in_second_label") == 1
)
print(data_in_second_label.count())
data_in_second_label.show(20,False)

## Rows with first and last name in third label

In [ ]:
data_in_third_label = first_and_last_name_in_root.select(
    psf.col("third_label"),
    psf.col("pii_in_third_label")
).distinct().filter(
    psf.col("pii_in_third_label") == 1
)
print(data_in_third_label.count())
data_in_third_label.show(20,False)

## Rows with first and last name in fourth label

In [ ]:
data_in_fourth_label = first_and_last_name_in_root.select(
    psf.col("fourth_label"),
    psf.col("pii_in_fourth_label")
).distinct().filter(
    psf.col("pii_in_fourth_label") == 1
)
print(data_in_fourth_label.count())
data_in_fourth_label.show(20,False)

## Rows with first and last name in fifth label

In [ ]:
data_in_fifth_label = first_and_last_name_in_root.select(
    psf.col("fifth_label"),
    psf.col("pii_in_fifth_label")
).distinct().filter(
    psf.col("pii_in_fifth_label") == 1
)
print(data_in_fifth_label.count())
data_in_fifth_label.show(20,False)

# Find the most-common used deepest labels in domain names

In [51]:
labels = df_dutch_domains_first_and_last_names.select(
    "domain",
    psf.regexp_extract(psf.col("domain"), "^([^.]+).+[.][^.]+$",1).alias("deepest-label")
).groupby(
    "deepest-label"
).agg(
    psf.count("domain").alias("domain_count")
).orderBy(psf.col("domain_count"), ascending=False)

labels.show(40, False)

+-------------+------------+
|deepest-label|domain_count|
+-------------+------------+
|www          |57565       |
|*            |14855       |
|mail         |9144        |
|webmail      |6160        |
|webdisk      |4633        |
|cpanel       |4628        |
|autodiscover |3256        |
|cpcontacts   |2452        |
|cpcalendars  |2452        |
|ipv6         |2092        |
|smtp         |700         |
|ftp          |692         |
|pop          |612         |
|preview      |559         |
|remote       |321         |
|m            |303         |
|test         |275         |
|builder      |210         |
|dev          |161         |
|whm          |158         |
|stack        |118         |
|cloud        |106         |
|webshop      |102         |
|bestellen    |100         |
|occasions    |99          |
|blog         |98          |
|staging      |93          |
|tools        |88          |
|new          |82          |
|nas          |75          |
|shop         |73          |
|portal       

In [52]:
# Save the deepest labels in a CSV file
labels.toPandas().to_csv("results/name={}/common_deepest_labels.csv".format(LOG))

## Save all results

In [53]:
# timestamp_format = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
df_dutch_domains_first_and_last_names.toPandas().to_csv("results/name={}/dutch_domain_with_pii_in_domain.csv".format(LOG))
most_occurring_root.toPandas().to_csv("results/name={}/dutch_domain_with_pii_in_domain.csv".format(LOG))
data_in_root.toPandas().to_csv("results/name={}/dutch_domain_with_pii_in_root.csv".format(LOG))
data_in_first_label.toPandas().to_csv("results/name={}/dutch_domain_with_pii_in_first_label.csv".format(LOG))
data_in_second_label.toPandas().to_csv("results/name={}/dutch_domain_with_pii_in_second_label.csv".format(LOG))
data_in_third_label.toPandas().to_csv("results/name={}/dutch_domain_with_pii_in_third_label.csv".format(LOG))
data_in_fourth_label.toPandas().to_csv("results/name={}/dutch_domain_with_pii_in_fourth_label.csv".format(LOG))

# Stop the SparkContext
Note: don't run this block unless you actually want to stop your context

In [12]:
print("[{}] Stopping Spark context.".format(datetime.now().replace(microsecond=0)))
# Commented to prevent accidental sequential execution
sc.stop()

[2021-06-24 19:55:26] Stopping Spark context.


# Obsolete
The code pieces below were used in the process, but became obsolete later in the process

# Filter out common deepest labels

The code below may seem illogical. Here is some explanation:
- First, we split the DF of domains in two parts: one part includes all domains that start with a common used label. The other part includes domains that do not start with a common used label.
- The part this includes domains with common used label is put through a regular expression that removes the deepest label and the first "." it occurs. For example, "mail.joe-doe.com" is replaced by "joe-doe.com".
- Afterwards, both parts are combined into one single DF containing all domain names.
- In the second code block, a distinct method is executed over the "domain" column (drop_duplicates()), resulting in a DF containing all unique domains.
- In the third code block, this DF is written to a local CSV file.

Note: feel free to uncomment the print statement to see how the domain names look at each state

In [23]:
## Load the top-25 most-common used deepest labels
common_used_deepest_labels = pd.read_csv("results/common_deepest_labels.csv")['deepest-label'].head(25)

## The two lines below replace ambiguous deepest labels to not exclude too many domains (for instance all domains that include "m." like "amsterdam.nl")
common_used_deepest_labels[12] = "inschrijven"

## Create an rlike string for most commonly used labels
label_string = '.|'.join(common_used_deepest_labels)+"."

## Filter data to only have domains with commonly used deepest label and remove the deepest label from the domain string
with_common_label = df_dutch_domains_first_and_last_names.filter(
    psf.col("domain").rlike(label_string).alias("domain")
)
# print("Here are the first domains with common label")
# with_common_label.show(20, False)
print("Number of domains with common label before removal",with_common_label.count())

## Create a new df with domains that do NOT start with a common label
without_common_label = df_dutch_domains_first_and_last_names.subtract(with_common_label)
# print("Here are the first domains with NO common label")
# without_common_label.show(20, False)
print("Number of domains without common label: ",without_common_label.count())

## Remove the common label from the domains that DO start with a common label
with_common_label_removed_label = with_common_label.select(
    psf.regexp_extract(psf.col("domain"), "^[^.]+.(.*)$",1).alias("domain"),
    "date"
)
# print("Here are the first domains with removed common label")
# with_common_label_removed_label.show(20, False)
print("Number of domains with common label after removal",with_common_label_removed_label.count())

## Combine the domains again
filtered_df = with_common_label_removed_label.union(without_common_label)
# print("Here are the first domains after joining")
# filtered_df.show(20, False)
print("Number of domains after combining",filtered_df.count())

Number of domains with common prefix before removal 1382171
Number of domains without common prefix:  728717
Number of domains with common prefix after removal 1382171
Number of domains after combining 2110888


In [ ]:
## Distinct the domains to only include unique domains
filtered_df_distinct = filtered_df.drop_duplicates(subset=['domain'])
print("Number of domains before distinct: ", filtered_df.count())
print("Number of domains after distinct: ", filtered_df_distinct.count())
# print("This were the first domains before distinct")
# filtered_df.show(100, False)
# print("This are the first domains after distinct")
# filtered_df_distinct.show(100, False)

In [25]:
## Store the distinct domains
timestamp_format = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
filtered_df_distinct.toPandas().to_csv("results/first_and_last_names/{}_{}m-records-first-and-last-names.csv".format(timestamp_format, str(NR_RECORDS//1000000)))

## Aggregate the total number of registered domains over time

In [5]:
df_domains = ct_raw_df.select(
    psf.explode('all_domains').alias("domain"),
    psf.col("leaf_cert.not_before").alias("date")
).select(
    psf.lower(psf.col("domain")).alias("domain"),
    psf.from_unixtime(psf.col("date"), "yyyy-MM-dd hh:mm").alias("date")
).groupby(
    psf.col("domain")
).agg(
    psf.min(psf.col("date")).alias("date")
)
# print("Print schema below:")
# filtered_df.printSchema()
# print("Size of df below:")
# df_dutch_domains.show(10,False)
print("The total number of rows is: ",df_domains.count())

The total number of rows is:  578720187


In [6]:
all_domains_agg_per_month = df_domains.withColumn("year", psf.year(psf.col("date"))).withColumn("month", psf.month(psf.col("date"))).select(
    psf.col("year"),
    psf.col("month"),
    psf.col("domain")
).groupBy(
    [psf.col("year"),psf.col("month")]
).count().repartition(1).orderBy(psf.col("year"),psf.col("month"))
# agg_per_month.show()
all_domains_agg_per_month.toPandas().to_csv("results/name={}/all_domain_agg_per_month.csv".format(LOG))

In [ ]:
early_domains = df_dutch_domains_first_and_last_names.withColumn("year", psf.year(psf.col("date"))).select(
    psf.col("domain"),
    psf.col("year")
).filter(
    psf.col("year")<2011
)
early_domains.show(100, False)

In [ ]:
very_old_dutch_domain = ct_raw_df.select(
    psf.explode('all_domains').alias("domain"),
    psf.col("leaf_cert.not_before").alias("before_date"),
    psf.col("leaf_cert.not_after").alias("after_date"),
).filter(
    psf.col("domain").contains('.nl')
).select(
    psf.lower(psf.col("domain")).alias("domain"),
    psf.from_unixtime(psf.col("before_date"), "yyyy-MM-dd hh:mm").alias("before_date"),
    psf.from_unixtime(psf.col("after_date"), "yyyy-MM-dd hh:mm").alias("after_date")
).groupby(
    psf.col("domain")
).agg(
    psf.min(psf.col("before_date")).alias("before_date"),
    psf.min(psf.col("after_date")).alias("after_date")
).filter(psf.col("domain").rlike(last_names_string)).filter(psf.col("domain").rlike(first_names_string)).withColumn("before_year", psf.year(psf.col("before_date"))).withColumn("after_year", psf.year(psf.col("after_date"))).select(
    psf.col("domain"),
    psf.col("before_year"),
    psf.col("after_year")
).filter(
    psf.col("before_year")<2008
)
very_old_dutch_domain.show(200, False)

In [ ]:
SEARCH_STRING = "cas.ms"
df_dutch_domains_first_and_last_names.filter(psf.col("domain").rlike(SEARCH_STRING)).show(100,False)

## Very old domains

In [10]:
very_old_domain = ct_raw_df.select(
    psf.explode('all_domains').alias("domain"),
    psf.col("leaf_cert.not_before").alias("before_date"),
    psf.col("leaf_cert.not_after").alias("after_date"),
).select(
    psf.lower(psf.col("domain")).alias("domain"),
    psf.from_unixtime(psf.col("before_date"), "yyyy-MM-dd hh:mm").alias("before_date"),
    psf.from_unixtime(psf.col("after_date"), "yyyy-MM-dd hh:mm").alias("after_date")
).groupby(
    psf.col("domain")
).agg(
    psf.min(psf.col("before_date")).alias("before_date"),
    psf.min(psf.col("after_date")).alias("after_date")
).withColumn("before_year", psf.year(psf.col("before_date"))).withColumn("after_year", psf.year(psf.col("after_date"))).select(
    psf.col("domain"),
    psf.col("before_year"),
    psf.col("after_year")
).filter(
    psf.col("before_year")<2000
)
very_old_domain.show(200, False)

+-------------------------------------------------------------+-----------+----------+
|domain                                                       |before_year|after_year|
+-------------------------------------------------------------+-----------+----------+
|*.securenow.com                                              |1999       |2000      |
|verisign class 1 public primary certification authority - g3 |1999       |2036      |
|hostway.com                                                  |1999       |2000      |
|globalsign primary class 2 ca                                |1999       |2009      |
|support.website2go.com                                       |1999       |2000      |
|netlock uzleti (class b) tanusitvanykiado                    |1999       |2019      |
|*.securesites.com                                            |1997       |1998      |
|eudranet cug ca                                              |1999       |2004      |
|utn-userfirst-hardware                    

In [16]:
vod_pandas = very_old_domain.toPandas()
vod_pandas_sort = vod_pandas.sort_values(['before_year'])

vod_pandas_sort

,domain,before_year,after_year
60,thawte premium server ca,1996,2021
64,thawte personal basic ca,1996,2021
56,thawte server ca dsa root,1996,2021
33,thawte server ca,1996,2021
13,gte cybertrust root,1996,2004
...,...,...,...
43,*.techsecure.com,1999,2000
41,http://www.valicert.com/,1999,2019
40,hitrust ca,1999,2014
54,btat2.www.wingspan.mybills.com,1999,2000


In [17]:
vod_pandas_sort.to_csv("results/name={}/old_domains.csv".format(LOG))